In [31]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

In [32]:
# Set up microsimulation for US households
sim = Microsimulation()

In [33]:
# Extract household data needed for rebate calculations
# Filter for California households only
state_code = sim.calculate("state_code_str", map_to="household")
ca_mask = state_code == "CA"

household_count_people = sim.calculate("household_count_people")[ca_mask]
household_adjusted_gross_income = sim.calculate("adjusted_gross_income", map_to="household")[ca_mask]
# Use snap_fpg for Federal Poverty Guidelines (this is the poverty threshold)
poverty_threshold = sim.calculate("snap_fpg", map_to="household")[ca_mask]
household_weight = sim.calculate("household_weight")[ca_mask]

print(f"Number of California households in sample: {len(household_count_people):,}")
print(f"Total weighted California households: {household_weight.sum():,.0f}")

Number of California households in sample: 1,854
Total weighted California households: 3,990,023,830,548


In [34]:
# INVESTIGATE WEIGHTING ISSUE
print("=== DEBUGGING HOUSEHOLD WEIGHTS ===")

# Check individual household weights
print(f"Sample of household weights: {household_weight[:10]}")
print(f"Min household weight: {household_weight.min():,.0f}")
print(f"Max household weight: {household_weight.max():,.0f}")
print(f"Mean household weight: {household_weight.mean():,.0f}")
print(f"Median household weight: {np.median(household_weight):,.0f}")

# Check if weights are reasonable
print(f"\nTotal weighted households: {household_weight.sum():,.0f}")
print(f"Number of household records: {len(household_weight):,}")
print(f"Average people per household: {household_count_people.mean():.1f}")

# Calculate implied population
total_weighted_people = (household_count_people * household_weight).sum()
print(f"Total weighted people: {total_weighted_people:,.0f}")
print(f"Expected CA population: ~39 million")
print(f"Weight scaling factor needed: {39e6 / total_weighted_people:.2e}")

# Check if we should use different weighting
# Try person weights instead of household weights
try:
    person_weight = sim.calculate("person_weight")[ca_mask.repeat(household_count_people)]
    print(f"\nPerson weights available - sample: {person_weight[:5]}")
except:
    print(f"\nPerson weights not available")

# Calculate proper household weights
# If total weighted people should = 39M, then household weights should be scaled
correct_total_people = 39e6
current_total_people = total_weighted_people
weight_correction_factor = correct_total_people / current_total_people
corrected_household_weight = household_weight * weight_correction_factor

print(f"\n=== CORRECTED WEIGHTS ===")
print(f"Weight correction factor: {weight_correction_factor:.2e}")
print(f"Corrected total households: {corrected_household_weight.sum()/1e6:.1f} million")
print(f"Corrected total people: {(household_count_people * corrected_household_weight).sum()/1e6:.1f} million")

=== DEBUGGING HOUSEHOLD WEIGHTS ===
Sample of household weights:            value         weight
0    1743.168091    1743.168091
1       0.122180       0.122180
2     756.053284     756.053284
3       0.220665       0.220665
4  358255.062500  358255.062500
5    2479.817871    2479.817871
6     269.325409     269.325409
7    4106.521484    4106.521484
8     291.358612     291.358612
9     386.776031     386.776031
Min household weight: 0
Max household weight: 680,607
Mean household weight: 253,075
Median household weight: 29

Total weighted households: 3,990,023,830,548
Number of household records: 1,854
Average people per household: 2.4
Total weighted people: 8,334,113,570,030
Expected CA population: ~39 million
Weight scaling factor needed: 4.68e-06

Person weights not available

=== CORRECTED WEIGHTS ===
Weight correction factor: 4.68e-06
Corrected total households: 18.7 million
Corrected total people: 39.0 million


In [35]:
# DEBUG: Why is average rebate $0?
print("=== DEBUGGING ZERO REBATE ISSUE ===")

print(f"Households with tax_free_consumption > 0: {(tax_free_consumption > 0).sum()}")
print(f"Sample tax_free_consumption values: {tax_free_consumption[tax_free_consumption > 0][:10]}")
print(f"Max tax_free_consumption: ${tax_free_consumption.max():,.0f}")
print(f"Mean tax_free_consumption (all): ${tax_free_consumption.mean():,.0f}")
print(f"Mean tax_free_consumption (>0 only): ${tax_free_consumption[tax_free_consumption > 0].mean():,.0f}")

# Check the masks
print(f"\nMask debugging:")
print(f"Households with income_to_poverty_ratio <= 1.5: {(income_to_poverty_ratio <= 1.5).sum()}")
print(f"Households with full_rebate_mask: {full_rebate_mask.sum()}")
print(f"Households with partial_rebate_mask: {partial_rebate_mask.sum()}")

# Check specific households that should get rebates - fix data type issues
low_income_mask = income_to_poverty_ratio <= 1.5
print(f"\nSample low-income households (should get full rebate):")

# Get first 5 indices where low_income_mask is True
low_income_indices = np.where(low_income_mask)[0][:5]

for i, idx in enumerate(low_income_indices):
    income_ratio = income_to_poverty_ratio.iloc[idx] if hasattr(income_to_poverty_ratio, 'iloc') else income_to_poverty_ratio[idx]
    poverty_val = poverty_threshold.iloc[idx] if hasattr(poverty_threshold, 'iloc') else poverty_threshold[idx]
    rebate_val = tax_free_consumption[idx]  # This is numpy array
    print(f"  HH {idx}: income_ratio={income_ratio:.2f}, poverty_threshold=${poverty_val:,.0f}, rebate=${rebate_val:,.0f}")

# The issue might be that tax_free_consumption is calculated correctly but we're looking at the wrong values
print(f"\nChecking rebate calculation step by step:")
print(f"Step 1 - Full rebate mask sum: {full_rebate_mask.sum()}")
print(f"Step 2 - Where full rebate applied: {np.where(full_rebate_mask, poverty_threshold, 0).sum()}")
print(f"Step 3 - Final tax_free_consumption sum: {tax_free_consumption.sum()}")

# Check data types
print(f"\nData type debugging:")
print(f"income_to_poverty_ratio type: {type(income_to_poverty_ratio)}")
print(f"poverty_threshold type: {type(poverty_threshold)}")
print(f"tax_free_consumption type: {type(tax_free_consumption)}")
print(f"household_weight type: {type(household_weight)}")

=== DEBUGGING ZERO REBATE ISSUE ===
Households with tax_free_consumption > 0: 502
Sample tax_free_consumption values: [19900.         25099.99804688 30300.         35500.00390625
 39799.99804688 30300.         14700.         19900.
 19900.         45000.        ]
Max tax_free_consumption: $86,600
Mean tax_free_consumption (all): $5,867
Mean tax_free_consumption (>0 only): $21,666

Mask debugging:
Households with income_to_poverty_ratio <= 1.5: 4927198.9318353
Households with full_rebate_mask: 4927198.9318353
Households with partial_rebate_mask: 61

Sample low-income households (should get full rebate):
  HH 1: income_ratio=0.29, poverty_threshold=$19,900, rebate=$19,900
  HH 3: income_ratio=0.24, poverty_threshold=$25,100, rebate=$25,100
  HH 4: income_ratio=0.30, poverty_threshold=$30,300, rebate=$30,300
  HH 6: income_ratio=0.97, poverty_threshold=$35,500, rebate=$35,500
  HH 8: income_ratio=1.43, poverty_threshold=$39,800, rebate=$39,800

Checking rebate calculation step by step:
St

In [36]:
# DEBUG: Specific issue with average rebate calculation
print("=== DETAILED REBATE CALCULATION DEBUG ===")

# Show households that should get rebates
rebate_households = tax_free_consumption > 0
print(f"Number of households with rebates: {rebate_households.sum()}")
print(f"Sample rebate amounts: {tax_free_consumption[rebate_households][:10]}")

# Calculate total rebate with corrected weights
total_rebate_with_corrected_weights = (tax_free_consumption[rebate_households] * household_weight_fixed[rebate_households]).sum()
total_households_with_rebates_corrected = household_weight_fixed[rebate_households].sum()

print(f"Total rebate amount (corrected weights): ${total_rebate_with_corrected_weights:,.0f}")
print(f"Total households receiving rebates (corrected weights): {total_households_with_rebates_corrected:,.0f}")

if total_households_with_rebates_corrected > 0:
    correct_average_rebate = total_rebate_with_corrected_weights / total_households_with_rebates_corrected
    print(f"Correct average rebate: ${correct_average_rebate:,.0f}")
else:
    print("No households receiving rebates - something is wrong!")

# Double-check the X calculation
X_corrected_check = (tax_free_consumption * household_weight_fixed).sum()
print(f"X (tax-free consumption) corrected: ${X_corrected_check/1e9:.1f} billion")

# Show some examples of households with actual rebate amounts - handle mixed data types
print(f"\n=== EXAMPLES OF HOUSEHOLDS WITH REBATES ===")
rebate_indices = np.where(rebate_households)[0][:10]  # Get first 10 indices with rebates

for idx in rebate_indices:
    # Handle mixed pandas/numpy data types properly
    income_ratio = income_to_poverty_ratio.iloc[idx] if hasattr(income_to_poverty_ratio, 'iloc') else income_to_poverty_ratio[idx]
    poverty_val = poverty_threshold.iloc[idx] if hasattr(poverty_threshold, 'iloc') else poverty_threshold[idx]
    rebate_val = tax_free_consumption[idx]  # numpy array
    print(f"  Household {idx}: Income/Poverty={income_ratio:.2f}, Poverty Threshold=${poverty_val:,.0f}, Rebate=${rebate_val:,.0f}")

=== DETAILED REBATE CALCULATION DEBUG ===
Number of households with rebates: 502
Sample rebate amounts: [19900.         25099.99804688 30300.         35500.00390625
 39799.99804688 30300.         14700.         19900.
 19900.         45000.        ]
Total rebate amount (corrected weights): $539,265
Total households receiving rebates (corrected weights): 6,052,438
Correct average rebate: $0
X (tax-free consumption) corrected: $0.0 billion

=== EXAMPLES OF HOUSEHOLDS WITH REBATES ===
  Household 1: Income/Poverty=0.29, Poverty Threshold=$19,900, Rebate=$19,900
  Household 3: Income/Poverty=0.24, Poverty Threshold=$25,100, Rebate=$25,100
  Household 4: Income/Poverty=0.30, Poverty Threshold=$30,300, Rebate=$30,300
  Household 6: Income/Poverty=0.97, Poverty Threshold=$35,500, Rebate=$35,500
  Household 8: Income/Poverty=1.43, Poverty Threshold=$39,800, Rebate=$39,800
  Household 9: Income/Poverty=1.06, Poverty Threshold=$30,300, Rebate=$30,300
  Household 13: Income/Poverty=0.00, Poverty 

In [37]:
# Debug: Check the values to understand why rebate is so low
print("=== DEBUGGING VALUES ===")
print(f"Sample poverty threshold values: {poverty_threshold[:10]}")
print(f"Sample AGI values: {household_adjusted_gross_income[:10]}")
print(f"Sample income to poverty ratios: {(household_adjusted_gross_income / poverty_threshold)[:10]}")
print(f"Mean poverty threshold: ${poverty_threshold.mean():,.2f}")
print(f"Median poverty threshold: ${np.median(poverty_threshold):,.2f}")
print(f"Mean AGI: ${household_adjusted_gross_income.mean():,.2f}")
print(f"Median AGI: ${np.median(household_adjusted_gross_income):,.2f}")

# Check if there are households that should get rebates
households_under_150_pct = (household_adjusted_gross_income / poverty_threshold <= 1.5).sum()
households_under_175_pct = (household_adjusted_gross_income / poverty_threshold <= 1.75).sum()
print(f"Households under 150% of poverty: {households_under_150_pct:,}")
print(f"Households under 175% of poverty: {households_under_175_pct:,}")

# Check for any zero or negative values that might cause issues
print(f"Zero poverty thresholds: {(poverty_threshold == 0).sum()}")
print(f"Zero AGI values: {(household_adjusted_gross_income == 0).sum()}")
print(f"Negative AGI values: {(household_adjusted_gross_income < 0).sum()}")

=== DEBUGGING VALUES ===
Sample poverty threshold values:           value         weight
0  14700.000000    1743.168091
1  19900.000000       0.122180
2  19900.000000     756.053284
3  25099.998047       0.220665
4  30300.000000  358255.062500
5  14700.000000    2479.817871
6  35500.003906     269.325409
7  40700.000000    4106.521484
8  39799.998047     291.358612
9  30300.000000     386.776031
Sample AGI values:            value         weight
0   42985.429688    1743.168091
1    5803.675293       0.122180
2  437602.093750     756.053284
3    6000.000000       0.220665
4    9000.679688  358255.062500
5   37174.089844    2479.817871
6   34400.679688     269.325409
7  118001.359375    4106.521484
8   56800.000000     291.358612
9   32200.000000     386.776031
Sample income to poverty ratios:        value         weight
0   2.924179    1743.168091
1   0.291642       0.122180
2  21.990055     756.053284
3   0.239044       0.220665
4   0.297052  358255.062500
5   2.528850    2479.817871
6

In [38]:
# Try different income measures to see which is appropriate
# Let's test which income variables exist
income_vars_to_test = [
    "household_net_income",
    "spm_unit_net_income", 
    "total_income",
    "employment_income"
]

print("=== TESTING AVAILABLE INCOME VARIABLES ===")
available_income_vars = {}

for var in income_vars_to_test:
    try:
        if "spm_unit" in var:
            income_vals = sim.calculate(var, map_to="household")[ca_mask]
        else:
            income_vals = sim.calculate(var, map_to="household")[ca_mask]
        available_income_vars[var] = income_vals
        print(f"{var}: Available, mean = ${income_vals.mean():,.2f}")
    except:
        print(f"{var}: Not available")

# Compare with AGI
print(f"\nAGI mean: ${household_adjusted_gross_income.mean():,.2f}")

# If we found alternatives, compare poverty ratios
if available_income_vars:
    print(f"\nHouseholds under 150% poverty by income measure:")
    agi_under_150 = (household_adjusted_gross_income / poverty_threshold <= 1.5).sum()
    print(f"Using AGI: {agi_under_150:,}")
    
    for var_name, income_vals in available_income_vars.items():
        under_150 = (income_vals / poverty_threshold <= 1.5).sum()
        print(f"Using {var_name}: {under_150:,}")

=== TESTING AVAILABLE INCOME VARIABLES ===
household_net_income: Available, mean = $116,398.37
spm_unit_net_income: Available, mean = $89,247.81
total_income: Not available
employment_income: Available, mean = $85,475.06

AGI mean: $132,662.72

Households under 150% poverty by income measure:
Using AGI: 4,927,198.9318353
Using household_net_income: 2,826,161.7897085254
Using spm_unit_net_income: 4,787,860.0594411995
Using employment_income: 6,631,240.491584323


In [39]:
# Analyze income distribution to understand why so few get rebates
print("=== INCOME TO POVERTY RATIO DISTRIBUTION ===")

income_to_poverty_ratio = household_adjusted_gross_income / poverty_threshold

# Show distribution of income-to-poverty ratios
percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    value = np.percentile(income_to_poverty_ratio, p)
    print(f"{p}th percentile income-to-poverty ratio: {value:.2f}")

print(f"\nPercent of households by income level:")
print(f"Below 50% of poverty: {(income_to_poverty_ratio <= 0.5).mean()*100:.1f}%")
print(f"Below 100% of poverty: {(income_to_poverty_ratio <= 1.0).mean()*100:.1f}%")
print(f"Below 150% of poverty: {(income_to_poverty_ratio <= 1.5).mean()*100:.1f}%")
print(f"Below 175% of poverty: {(income_to_poverty_ratio <= 1.75).mean()*100:.1f}%")
print(f"Below 200% of poverty: {(income_to_poverty_ratio <= 2.0).mean()*100:.1f}%")

# Check if the issue is with zero/negative incomes
zero_income_mask = household_adjusted_gross_income <= 0
print(f"\nHouseholds with zero or negative AGI: {zero_income_mask.sum():,} ({zero_income_mask.mean()*100:.1f}%)")

# If there are many zero-income households, they might be skewing results
if zero_income_mask.sum() > 0:
    print("Income distribution excluding zero/negative income households:")
    positive_income_mask = household_adjusted_gross_income > 0
    positive_income_ratio = household_adjusted_gross_income[positive_income_mask] / poverty_threshold[positive_income_mask]
    
    for p in [10, 25, 50, 75, 90]:
        value = np.percentile(positive_income_ratio, p)
        print(f"{p}th percentile (positive income only): {value:.2f}")
    
    print(f"Below 150% of poverty (positive income only): {(positive_income_ratio <= 1.5).mean()*100:.1f}%")
    print(f"Below 175% of poverty (positive income only): {(positive_income_ratio <= 1.75).mean()*100:.1f}%")

=== INCOME TO POVERTY RATIO DISTRIBUTION ===
10th percentile income-to-poverty ratio: 0.32
25th percentile income-to-poverty ratio: 1.59
50th percentile income-to-poverty ratio: 3.50
75th percentile income-to-poverty ratio: 6.68
90th percentile income-to-poverty ratio: 12.48
95th percentile income-to-poverty ratio: 19.53
99th percentile income-to-poverty ratio: 46.38

Percent of households by income level:
Below 50% of poverty: 18.9%
Below 100% of poverty: 24.6%
Below 150% of poverty: 31.3%
Below 175% of poverty: 33.5%
Below 200% of poverty: 37.3%

Households with zero or negative AGI: 1,426,160.1974516786 (9.0%)
Income distribution excluding zero/negative income households:
10th percentile (positive income only): 0.85
25th percentile (positive income only): 1.87
50th percentile (positive income only): 3.74
75th percentile (positive income only): 6.94
90th percentile (positive income only): 12.86
Below 150% of poverty (positive income only): 24.4%
Below 175% of poverty (positive income

In [40]:
# Calculate rebate with phase-out logic using weighted calculations
# Rebate = poverty_threshold for households below 150% of poverty threshold
# Phases out linearly between 150% and 175% of poverty threshold
# No rebate for households above 175% of poverty threshold

# Define phase-out thresholds
phase_out_start = 1.5  # 150% of poverty threshold
phase_out_end = 1.75   # 175% of poverty threshold

# Calculate income as percentage of poverty threshold
income_to_poverty_ratio = household_adjusted_gross_income / poverty_threshold

# Calculate tax-free consumption (X component) for each household
# Full poverty threshold amount for households below 150% of poverty
full_rebate_mask = income_to_poverty_ratio <= phase_out_start
tax_free_consumption = np.where(full_rebate_mask, poverty_threshold, 0)

# Partial rebate for households between 150% and 175% of poverty
partial_rebate_mask = (income_to_poverty_ratio > phase_out_start) & (income_to_poverty_ratio <= phase_out_end)
phase_out_factor = (phase_out_end - income_to_poverty_ratio) / (phase_out_end - phase_out_start)
partial_tax_free = poverty_threshold * phase_out_factor
tax_free_consumption = np.where(partial_rebate_mask, partial_tax_free, tax_free_consumption)

# Count weighted households by rebate status
full_rebate_households = (household_weight[full_rebate_mask & (tax_free_consumption > 0)]).sum()
partial_rebate_households = (household_weight[partial_rebate_mask & (tax_free_consumption > 0)]).sum()
total_rebate_households = full_rebate_households + partial_rebate_households

print(f"Weighted households receiving full rebate: {full_rebate_households/1e6:.1f} million")
print(f"Weighted households receiving partial rebate: {partial_rebate_households/1e6:.1f} million")
print(f"Total weighted households receiving any rebate: {total_rebate_households/1e6:.1f} million")

Weighted households receiving full rebate: 1270717.1 million
Weighted households receiving partial rebate: 22659.9 million
Total weighted households receiving any rebate: 1293377.1 million


In [41]:
# FINAL CORRECTED RESULTS using proper calculation
household_weight_fixed = corrected_household_weight

print("=== FINAL CORRECTED REBATE ANALYSIS ===")

# Recalculate tax-free consumption with corrected weights
X_corrected = (tax_free_consumption * household_weight_fixed).sum()

# Get households that receive rebates
rebate_recipients = tax_free_consumption > 0

# Calculate statistics for rebate recipients only
full_rebate_recipients = full_rebate_mask & rebate_recipients
partial_rebate_recipients = partial_rebate_mask & rebate_recipients

full_rebate_households_corrected = household_weight_fixed[full_rebate_recipients].sum()
partial_rebate_households_corrected = household_weight_fixed[partial_rebate_recipients].sum()
total_rebate_households_corrected = household_weight_fixed[rebate_recipients].sum()

print(f"Households receiving full rebate: {full_rebate_households_corrected/1e6:.1f} million")
print(f"Households receiving partial rebate: {partial_rebate_households_corrected/1e6:.1f} million") 
print(f"Total households receiving any rebate: {total_rebate_households_corrected/1e6:.1f} million")

# Calculate correct average rebate
total_rebate_amount = (tax_free_consumption[rebate_recipients] * household_weight_fixed[rebate_recipients]).sum()
if total_rebate_households_corrected > 0:
    average_rebate_corrected = total_rebate_amount / total_rebate_households_corrected
else:
    average_rebate_corrected = 0

# Summary statistics
total_weighted_households_corrected = household_weight_fixed.sum()

print(f"\n=== SUMMARY ===")
print(f"Total CA households: {total_weighted_households_corrected/1e6:.1f} million")
print(f"Households receiving rebate: {total_rebate_households_corrected/1e6:.1f} million ({100*total_rebate_households_corrected/total_weighted_households_corrected:.1f}%)")
print(f"Average rebate amount: ${average_rebate_corrected:,.0f}")
print(f"X (total tax-free consumption): ${X_corrected/1e9:.1f} billion")
print(f"Total rebate cost: ${total_rebate_amount/1e9:.1f} billion")

print(f"\n=== FOR VAT FORMULA ===")
print(f"X = ${X_corrected:,.0f}")
print(f"This represents the total amount of tax-free consumption")
print(f"Used in VAT rate calculation: t = Rs/(Cp - X - T + Ro)")

=== FINAL CORRECTED REBATE ANALYSIS ===
Households receiving full rebate: 5.9 million
Households receiving partial rebate: 0.1 million
Total households receiving any rebate: 6.1 million

=== SUMMARY ===
Total CA households: 18.7 million
Households receiving rebate: 6.1 million (32.4%)
Average rebate amount: $0
X (total tax-free consumption): $0.0 billion
Total rebate cost: $0.0 billion

=== FOR VAT FORMULA ===
X = $539,265
This represents the total amount of tax-free consumption
Used in VAT rate calculation: t = Rs/(Cp - X - T + Ro)


In [42]:
# Summary statistics with corrected calculations
total_weighted_households = household_weight.sum()
weighted_households_with_rebate = (household_weight[tax_free_consumption > 0]).sum()

# Calculate average rebate among households receiving any rebate
rebate_mask = tax_free_consumption > 0
if rebate_mask.sum() > 0:
    total_rebate_amount = (tax_free_consumption[rebate_mask] * household_weight[rebate_mask]).sum()
    weighted_households_receiving_rebate = household_weight[rebate_mask].sum()
    average_rebate = total_rebate_amount / weighted_households_receiving_rebate
else:
    average_rebate = 0
    weighted_households_receiving_rebate = 0

print("=== CORRECTED SUMMARY STATISTICS ===")
print(f"Total weighted CA households: {total_weighted_households/1e6:.1f} million")
print(f"Weighted households receiving any rebate: {weighted_households_receiving_rebate/1e6:.1f} million")
print(f"Percentage of households receiving rebate: {100*weighted_households_receiving_rebate/total_weighted_households:.1f}%")
print(f"Average rebate amount: ${average_rebate:,.0f}")
print(f"X (total tax-free consumption): ${X/1e9:.1f} billion")

# Validate against expected California household count (~13-15 million)
expected_ca_households = 13.5e6  # Approximate CA household count
scaling_factor = expected_ca_households / total_weighted_households
print(f"\n=== SCALED TO REALISTIC CA HOUSEHOLD COUNT ===")
print(f"Scaling factor: {scaling_factor:.2e}")
print(f"Scaled households with rebate: {weighted_households_receiving_rebate * scaling_factor / 1e6:.1f} million")
print(f"Scaled X (tax-free consumption): ${X * scaling_factor / 1e9:.1f} billion")

=== CORRECTED SUMMARY STATISTICS ===
Total weighted CA households: 3990023.8 million
Weighted households receiving any rebate: 1293377.1 million
Percentage of households receiving rebate: 32.4%
Average rebate amount: $0
X (total tax-free consumption): $115.2 billion

=== SCALED TO REALISTIC CA HOUSEHOLD COUNT ===
Scaling factor: 3.38e-06
Scaled households with rebate: 4.4 million
Scaled X (tax-free consumption): $0.0 billion
